### Specify Inputs

In [1]:
#Define Batches
batches = "UCB003A[E-J]"#|UCB003AE-US00-24|UCB003AF-US00-24|UCB003AF-US00-26|UCB003AG-US00-02|UCB003AG-US00-15|UCB003AG-US00-34|UCB003AG-US00-35|"


# Define Layers
Layers = 3  # Number of samples in each group

### Import Modules and Functions

In [2]:
#import modules used in this script
import pandas as pd
import numpy as np
import genealogy_v2
import query_tray_samples_V4 as query_tray_samples
import unit_cell_electrical_yield_and_metrics_v2 as uceym
import warnings
from datetime import datetime
from qsdc.client import Client
#assert version("qs-data-client") >= "0.2.15", "qsdc version must be greater than 0.2.15"

#Remove warning messages
warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)
#establish clients 
qs_client = Client()
conn = qs_client.get_mysql_engine()

# !pip install --upgrade qs-data-client  ## RUN THIS IF YOU NEED TO UPDATE qs-data-client
#Run in terminal if not already logged in:     gcloud auth application-default login

c:\Users\EDM02\OneDrive - QuantumScape Corporation\Files\Python Scripts\UnitCellTiering_Git\genealogy_v2.py:533: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  pivoted_df[column_name] = None
c:\Users\EDM02\OneDrive - QuantumScape Corporation\Files\Python Scripts\UnitCellTiering_Git\genealogy_v2.py:533: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  pivoted_df[column_name] = None
c:\Users\EDM02\OneDrive - QuantumScape Corporation\Files\Python Scripts\UnitCellTiering_Git\genealogy_v2.py:533: PerformanceWarning: DataFrame is highly f

### Pull Cell Data From Datahub

The two datasets that we care about are:
* `MFG-60L-UC-CTQ`. This should Critical to Quality metrics per the <u>[specs](https://qs-cloud.atlassian.net/wiki/spaces/HAB/pages/1782611969/Specifications+A1+Cell+Tiering+-+Post+UCT)</u> provided by Jordi. It contains columns `disposition` and `failure_modes` which should be pretty self-explanatory.
* `MFG-60L-UC-MR`. This table should contain the same number of rows (one per cell) as the CTQ table, but the `disposition_mr` and `failure_modes_mr` columns will be replaced with values corresponding to the manual review data. If no manual review was done, the standard metro `diposition` and `failure_modes` will be reflected.This makes it much easier to do 1:1 comparisons across the datasets. We can adjust to include only the manually reviewed cells if that's the preference.


In [3]:
## Query Cell Metrology and Electrical Data

#Pull cell metrology data from datahub, both standard/auto metrology and manual review
dfctq = qs_client.data_hub.get_dataset(dataset = 'MFG-60L-UC-CTQ') ## standard metro
dfmr = qs_client.data_hub.get_dataset(dataset = 'MFG-60L-UC-MR') ## manual review
dfc = qs_client.data_hub.get_dataset(dataset = 'MFG-60L-UNIT-CELL-TEST-CYCLE') ##electrical test data
#dfc = dfc[dfc['US_id'].str.contains(batches, na=False)]
cols = [
    "US_id",
    "TestCycleStart_datetime_first",
    "TestCycleStart_datetime",
    "UCT_Version",
    "idtest_recipe",
    "RunIndex",
    "CycleIndex",
    "dvdt",
    "recipe_dvdt_range",
    "CeilingHoldTime",
    "CE",
    "CapacityChargeFraction",
    "CeilingRestVoltage",
    "AMSDcCapacity",
    "CycleFailure",
    "AnyFailure",
    "MedDcASR",
    "DischargeCapacity"

]
dfc = dfc[dfc['US_id'].str.contains(batches, na=False)].sort_values(["RunIndex","CycleIndex"])[cols]

# Keep cells from batches of interest and remove those that failed screening
filtered_dfctq = dfctq[dfctq['US_id'].str.contains(batches, na=False)] #keep cells that are from the batches of interest
yielded_dfctq = filtered_dfctq[filtered_dfctq['unit_cell_test_yield'] == 1] #keep cells that yielded
filtered_dfmr = dfmr[dfmr['US_id'].str.contains(batches, na=False)] #keep only cells that are from the batches of interest
yielded_dfmr = filtered_dfmr[filtered_dfmr['unit_cell_test_yield'] == 1] #keep cells that yielded

c:\Users\EDM02\AppData\Local\Programs\Python\Python39\lib\site-packages\qsdc\datahub\datahub.py:88: DtypeWarning: Columns (30,31,32,33,35,36,75) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(bytesIO(blob_bytes))


### Generate Spreadsheet

In [4]:
## Overwrite automated results with manual review results
# First, merge the DataFrames on 'US_id' to align rows
merged_df = yielded_dfctq.merge(yielded_dfmr[['US_id', 'edge_thickness_tier_us_mr', 'A1_anode_tier_top_us_mr', 'A1_anode_tier_bottom_us_mr',
                                              'cathode_alignment_custom_model_tier_us_mr', 'median_contour_catholyte_pct_us_mr', 'max_f2f_distance_us','disposition_mr', 'failure_modes_mr']], on='US_id', how='left')
# Then, overwrite 'edge_thickness_tier_us' in 'filtered_dfctq' where 'edge_thickness_tier_us_mr' has a value
merged_df['edge_thickness_tier_us'] = merged_df['edge_thickness_tier_us_mr'].combine_first(merged_df['edge_thickness_tier_us'])
# Then, overwrite 'A1_anode_tier_top_us' in 'filtered_dfctq' where 'A1_anode_tier_top_us_mr' has a value
merged_df['A1_anode_tier_top_us'] = merged_df['A1_anode_tier_top_us_mr'].combine_first(merged_df['A1_anode_tier_top_us'])
# Then, overwrite 'A1_anode_tier_bottom_us' in 'filtered_dfctq' where 'A1_anode_tier_bottom_us_mr' has a value
merged_df['A1_anode_tier_bottom_us'] = merged_df['A1_anode_tier_bottom_us_mr'].combine_first(merged_df['A1_anode_tier_bottom_us'])
# Then, overwrite 'A1_anode_tier_bottom_us' in 'filtered_dfctq' where 'A1_anode_tier_bottom_us_mr' has a value
merged_df['cathode_alignment_custom_model_tier_us'] = merged_df['cathode_alignment_custom_model_tier_us_mr'].combine_first(merged_df['cathode_alignment_custom_model_tier_us'])
# Then, overwrite 'A1_anode_tier_bottom_us' in 'filtered_dfctq' where 'A1_anode_tier_bottom_us_mr' has a value
merged_df['median_contour_catholyte_pct_us'] = merged_df['median_contour_catholyte_pct_us_mr'].combine_first(merged_df['median_contour_catholyte_pct_us'])
# Then, overwrite 'disposition_us' in 'disposition_mr' has a value
merged_df['disposition'] = merged_df['disposition_mr'].combine_first(merged_df['disposition'])
# Then, overwrite 'disposition_us' in 'disposition_mr' has a value
merged_df['failure_modes'] = merged_df['failure_modes_mr'].combine_first(merged_df['failure_modes'])
# Drop the 'edge_thickness_tier_us_mr' column if you don't need it
dfctq_updated = merged_df.drop(columns=['edge_thickness_tier_us_mr', 'A1_anode_tier_top_us_mr', 'A1_anode_tier_bottom_us_mr',
                                              'cathode_alignment_custom_model_tier_us_mr', 'median_contour_catholyte_pct_us_mr', 'disposition_mr','failure_modes_mr' ])

#Update Final Tier of Cells
conditions = [
    dfctq_updated['disposition'] == 'Tier 1',
    dfctq_updated['disposition'] == 'Tier 2',
    dfctq_updated['disposition'] == 'Fail',
    dfctq_updated['disposition'] == 'Scrap',
    dfctq_updated['disposition'] == 'Missing Data',
]
choices = ['1', '2', '3','Scrapped', 'TBD']
dfctq_updated['Tier'] = np.select(conditions, choices)

In [5]:
### Fill in other columns of spreadsheet

## Rename Spreadsheet and sample/batch columns
CellTiering = dfctq_updated[['US_id']].rename(columns={'US_id': 'Cell ID'}) 
CellTiering['Batch'] = dfctq_updated['US_process_flow'] #Create Tiering Spreadsheet
# Create columns for final spreadsheet
new_columns = ["Cell Status", "Cell Tier", "Edge Wetting", "Thickness", "Alignment", "Anode","Film-to-Film Distance"]
for col in new_columns:
    CellTiering[col] = np.nan


##Update Cell Tiering metrics in final spreadsheet
# Merge the dfctq_updated with the CellTiering
merged_df = CellTiering.merge(
    dfctq_updated[['US_id', 'Tier', 'median_contour_catholyte_pct_us', 'edge_thickness_tier_us', 'cathode_alignment_custom_model_tier_us', 'A1_anode_tier_top_us','A1_anode_tier_bottom_us', 'max_f2f_distance_us']],
    left_on='Cell ID',
    right_on='US_id',
    how='left'
)
# Update final 'Cell Tier' columns with data  from dfctq_updated
merged_df['Cell Tier'] = merged_df['Tier']
#Update 'Edge Wetting' column
EWconditions = [
    merged_df['median_contour_catholyte_pct_us'] < 80,
    (merged_df['median_contour_catholyte_pct_us'] >= 80) & (merged_df['median_contour_catholyte_pct_us'] <= 98),
    merged_df['median_contour_catholyte_pct_us'] > 98
]
EWchoices = [3, 2, 1]
merged_df['Edge Wetting'] = np.select(EWconditions, EWchoices)
#Update 'Thickness' Column
merged_df['Thickness'] = merged_df['edge_thickness_tier_us']
#Update Alignment' Column
merged_df['Alignment'] = merged_df['cathode_alignment_custom_model_tier_us']
#Update 'Anode' Column
merged_df['Anode'] = np.maximum(merged_df['A1_anode_tier_top_us'], merged_df['A1_anode_tier_bottom_us'])
#Update 'Film-to-Film Distance Column
#merged_df['Film-to-Film Distance'] = 'Low'  # Default value
merged_df.loc[merged_df['max_f2f_distance_us'] > 0.40, 'Film-to-Film Distance'] = 'High'
# Drop the extra columns from dfctq_updated
CellTiering = merged_df.drop(columns=['US_id', 'Tier', 'median_contour_catholyte_pct_us', 'edge_thickness_tier_us','cathode_alignment_custom_model_tier_us','A1_anode_tier_top_us','A1_anode_tier_bottom_us','max_f2f_distance_us'])


##Update 'Cell Status' if yielded cell is on already in a multilayer pouch
gen = genealogy_v2.get_genealogy_2L('APD|ML|UC|QSC', conn) # Query dataframe from database
us_to_ps_mapping = dict(zip(gen['2L_cell_id'], gen['6L_cell_id'])) # Map the 'US_id' to '6L_cell_id' using '2L_cell_id' in the 'gen' dataframe
CellTiering['Cell Status'] = CellTiering['Cell ID'].map(us_to_ps_mapping)
CellTiering['Cell Status'] = CellTiering['Cell Status'].fillna('Waiting')


## Update cell electrical metrics, currently using python scripts to query but will need to transition to datahub using dataframe dfc
df_electrical_yield_metrics = uceym.get_electrical_yield_and_metrics(CellTiering["Cell ID"])
df_electrical_yield_metrics = df_electrical_yield_metrics[['US_id','MedDischargeASR_1C', 'DischargeCapacity_Co3', 'dVdt_delta_fastcharge']]
df_electrical_yield_metrics = df_electrical_yield_metrics.rename(columns={'US_id': 'Cell ID'}) #Create Tiering Spreadsheet
CellTiering = pd.merge(CellTiering, df_electrical_yield_metrics, left_on='Cell ID', right_on='Cell ID', how='left')


##Calculate Cell Tiering Score
ml_features = [
        'MedDischargeASR_1C',
        'DischargeCapacity_Co3',
        'dVdt_delta_fastcharge',
        'Cell Tier',
    ]
ranking_weights = [
        0.10, 
        0.00, 
        0.00, 
        0.90
        ]
for feature in ml_features:
    CellTiering[feature] = pd.to_numeric(CellTiering[feature], errors='coerce')        
CellTiering['total_rank_score'] = (
    CellTiering[ml_features[0]] * ranking_weights[0] +
    CellTiering[ml_features[1]] * ranking_weights[1] +
    CellTiering[ml_features[2]] * ranking_weights[2] +
    CellTiering[ml_features[3]] * ranking_weights[3]
)


## Update tray location ##
sample_names_group = CellTiering["Cell ID"]
df_tray = query_tray_samples.get_sample_tray(sample_names_group)
CellTiering = pd.merge(CellTiering, df_tray[['sample_name', 'tray_id', 'row_index', 'col_index', 'modified']], left_on='Cell ID', right_on='sample_name', how='left')
CellTiering = CellTiering.drop(columns=['sample_name', 'modified'])

#Update if yielded cell is on 2L Reliability
#data = data[["Cell ID","Reliability Test Count"]]
# Merge the two dataframes on the 'Cell ID' column
#merged_df = pd.merge(CellTiering, data, on='Cell ID', how='right')
# Update 'Cell Status' where 'Reliability Test Count' is 1 in merged dataframe
#merged_df.loc[merged_df['Reliability Test Count'] == 1, 'Cell Status'] = '2L Reliability'
# Drop the added columns to keep only the original columns from YieldedCells
#YieldedCells = merged_df.drop(columns=['Reliability Test Count'])




In [6]:
##save Tiering spreadsheet
current_date = datetime.now().strftime('%Y-%m-%d')
output_name = f'{current_date}_Tiered.xlsx'
CellTiering.to_excel(output_name, index=False)


In [7]:
## Pair cells and save Tiered&Paired spreadsheet

df = CellTiering
df = df[df['Cell Status'] == 'Waiting'] # Filter rows where "Cell Status" is 'Waiting'
LargeF2F = df[df['Film-to-Film Distance'] == 'High']
LargeF2F['Group ID'] = 'Bad F2F Alignment'
LargeF2F = LargeF2F.sort_values(by='total_rank_score', ascending=True)

df = df[df['Film-to-Film Distance'] != 'High'] # Filter out rows where film-to-film is 'High'



# Group samples by "Pairing Group" and form groups by "Layers"
grouped_samples = []

for pairing_group, group_df in df.groupby('Cell Tier'):
    group_id = 1  # To uniquely identify each full group
    # Sort samples by "MedDischargeASR_1C" within each "Pairing Group"
    sorted_group = group_df.sort_values(by='MedDischargeASR_1C').reset_index(drop=True)
    
    # Divide into groups of size "Layers"
    num_groups = int(np.ceil(len(sorted_group) / Layers))
    

    for i in range(num_groups):
        # Extract the i-th group (slice of the sorted DataFrame)
        group_chunk = sorted_group.iloc[i * Layers: (i + 1) * Layers].copy()
        
        # Check if the chunk has enough samples
        if len(group_chunk) < Layers:
            # Assign "Backfill" ID if it's a partial group
            group_chunk['Group ID'] = f"Backfill Tier {pairing_group}"
        else:
            # Assign a unique group identifier for full groups
            group_chunk['Group ID'] = f"Group {pairing_group}_{group_id}"
            group_id += 1
        
        grouped_samples.append(group_chunk)

# Combine all grouped samples into a single DataFrame
GroupedCells = pd.concat(grouped_samples).reset_index(drop=True)
# Combine the DataFrames
GroupedCells = pd.concat([GroupedCells,LargeF2F], ignore_index=True)
GroupedCells = GroupedCells.drop(columns=['Film-to-Film Distance'])
# Sort the combined DataFrame in descending order by "Group ID"
#GroupedCells= GroupedCells.sort_values(by='Group ID', ascending=False)

GroupedCells.to_clipboard(index=False)

##save spreadsheet
G_output_name = f'{current_date}_Tiered&Paired.xlsx'
GroupedCells.to_excel(G_output_name, index=False)



C:\Users\EDM02\AppData\Local\Temp\ipykernel_24060\2223520024.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LargeF2F['Group ID'] = 'Bad F2F Alignment'


In [8]:
dfc

,US_id,TestCycleStart_datetime_first,TestCycleStart_datetime,UCT_Version,idtest_recipe,RunIndex,CycleIndex,dvdt,recipe_dvdt_range,CeilingHoldTime,CE,CapacityChargeFraction,CeilingRestVoltage,AMSDcCapacity,CycleFailure,AnyFailure,MedDcASR,DischargeCapacity
219024,UCD006AY-US00-01,2024-11-13 16:08:50,2024-11-13 16:08:50,Y2_A1_V3,14212,2,1,-13.693425,0.0,5411.82,0.808104,1.173350,4.21791,183.949,False,True,25.4330,471.681
219031,UCD006AY-US00-02,2024-11-13 16:08:53,2024-11-13 16:08:53,Y2_A1_V3,14212,2,1,-14.044909,0.0,5396.17,0.806771,1.173440,4.21770,183.659,False,True,25.5597,470.967
219037,UCD006AY-US00-03,2024-11-13 16:08:58,2024-11-13 16:08:58,Y2_A1_V3,14212,2,1,-28.194701,0.0,7199.95,0.671946,1.384020,4.16107,180.417,True,True,24.9100,462.973
219039,UCD006AY-US00-04,2024-11-13 16:08:59,2024-11-13 16:08:59,Y2_A1_V3,14212,2,1,-13.923424,0.0,5448.45,0.807591,1.174460,4.21768,184.006,False,False,25.1930,472.143
219052,UCD006AY-US00-05,2024-11-13 16:09:03,2024-11-13 16:09:03,Y2_A1_V3,14212,2,1,-14.005084,0.0,5461.87,0.806302,1.170410,4.21760,183.079,False,True,25.8526,470.311
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224656,UCD011AL-US00-24,2024-11-13 16:05:34,2024-11-15 04:46:37,Y2_A1_V3,13711,7,1,-0.541731,0.0,2392.98,NaN,0.312344,3.73986,NaN,False,False,NaN,NaN
224677,UCD011AL-US00-27,2024-11-13 16:05:44,2024-11-15 04:32:30,Y2_A1_V3,13711,7,1,-0.532698,0.0,2279.70,NaN,0.315943,3.74037,NaN,False,False,NaN,NaN
224690,UCD011AL-US00-28,2024-11-13 16:05:46,2024-11-15 04:29:11,Y2_A1_V3,13711,7,1,-0.603710,0.0,2265.69,NaN,0.314043,3.74034,NaN,False,False,NaN,NaN
224703,UCD011AL-US00-29,2024-11-13 16:05:50,2024-11-15 04:35:52,Y2_A1_V3,13711,7,1,-0.333209,0.0,2312.16,NaN,0.314313,3.74028,NaN,False,False,NaN,NaN
